In [1]:
import torch
import pandas as pd
import random
import numpy as np
torch.cuda.is_available()
torch.cuda.get_device_name(0)

'NVIDIA GeForce RTX 3080 Laptop GPU'

In [2]:
en_gez = pd.read_csv('en_gez.csv')

In [3]:
en_gez.head()

,source_lang,target_lang
0,"And they took their journey from Elim, and all...",ወግዕዙ እምኤሌም ወመጽኡ ኵሉ ደቂቀ እስራኤል ውስተ ገዳም ዘሲን ወውእቱ ...
1,"And it came to pass, as Aaron spake unto the w...",ወሶበ ይትናገር አሮን ለኵሉ ኀበ ተጋብኡ ውሉደ እስራኤል ወተመይጡ ውስተ ...
2,"And the LORD spake unto Moses, saying,",ወይቤ እግዚአብሔር ለሙሴ ፤
3,I have heard the murmurings of the children of...,ሰማዕኩ ነጐርጓሮሙ ለውሉደ እስራኤል ወበሎሙ ሰርከ ትበልዑ ሥጋ ወጸቢሖ ት...
4,"And it came to pass, that at even the quails c...",ወመስየ ወመጽአ ፍርፍርት ወከደነ ኵሎ ትዕይንቶሙ ወነግሀ እንዘ የኀድግ ህ...


In [4]:
len(en_gez)

11810

In [5]:
random.seed(41)

In [6]:
en_gez_shuffled = en_gez.sample(frac=1, random_state=41).reset_index(drop=True)

In [7]:
en_gez_shuffled.head()

,source_lang,target_lang
0,"Thou makest us a reproach to our neighbours, a...",ወወሀብከነ ይብልዑነ ከመ አባግዕ፤ ወዘረውከነ ውስተ አሕዛብ።
1,"And the king said unto Joab, Behold now, I hav...",ወይቤሎ ንጉሥ ለኢዮአብ ናሁ ገበርኩ ለከ በከመ ዝንቱ ነገርከ ፤ አእትዎ ...
2,"And the elders of Gilead said unto Jephthah, T...",ወይቤልዎ ሊቃናተ ገላአድ ለይፍታሔ አኮ ከመዝ ፤ መጻእነ ኀቤከ ከመ ትሑር...
3,But unto the place which the LORD your God sha...,ዘእንበለ ውስተ መካን ዘኀረየ እግዚአብሔር በአሐቲ እምነ አህጉሪክሙ ከመ ...
4,"And it came to pass after this, that there was...",ወእምዝ ኮነ ዓዲ ፀብእ ውስተ ጌት ምስለ ኢሎፍሊ ይእተ አሚረ ቀተሎ ሴሜኮ...


In [8]:
size = len(en_gez_shuffled)
train_size = int(size*0.70)
test_size = int(size*0.20)
val_size = int(size*0.10)
en_train_data = en_gez_shuffled['source_lang'][:train_size]
en_test_data = en_gez_shuffled['source_lang'][train_size:train_size+test_size]
en_val_data = en_gez_shuffled['source_lang'][train_size+test_size: ]

gez_train_data = en_gez_shuffled['target_lang'][:train_size]
gez_test_data = en_gez_shuffled['target_lang'][train_size:train_size+test_size]
gez_val_data = en_gez_shuffled['target_lang'][train_size+test_size: ]

In [52]:
np.savetxt('data/en_train.txt', en_train_data.values, fmt='%s')
np.savetxt('data/en_test.txt', en_test_data.values, fmt='%s')
np.savetxt('data/en_val.txt', en_val_data.values, fmt='%s')

np.savetxt('data/gez_train.txt', gez_train_data.values, fmt='%s')
np.savetxt('data/gez_test.txt', gez_test_data.values, fmt='%s')
np.savetxt('data/gez_val.txt', gez_val_data.values, fmt='%s')

In [2]:
config = '''
## Where the samples will be written
save_data: data/

## Where the vocab(s) will be written
src_vocab: data/en.vocab.src
tgt_vocab: data/gez.vocab.tgt

# Corpus opts:
data:
    corpus_1:
        path_src: data/en_train.txt
        path_tgt: data/gez_train.txt
    valid:
        path_src: data/en_val.txt
        path_tgt: data/gez_val.txt


## Where the model will be saved
save_model: model/model

# Prevent overwriting existing files in the folder
overwrite: True


# General opts
save_model: mybasemodel
save_checkpoint_steps: 500
valid_steps: 500
train_steps: 20000
seed: 3435

# Batching
bucket_size: 262144
world_size: 1
gpu_ranks: [0]
num_workers: 2
batch_type: "tokens"
batch_size: 1024
valid_batch_size: 128
accum_count: [4]
accum_steps: [0]

# Optimization
model_dtype: "fp16"
optim: "adam"
learning_rate: 0.1
warmup_steps: 1000
decay_method: "noam"
adam_beta2: 0.998
max_grad_norm: 0
label_smoothing: 0.1
param_init: 0
param_init_glorot: true
normalization: "tokens"

early_stopping: 5
early_stopping_criteria : accuracy

# Model
encoder_type: transformer
decoder_type: transformer
position_encoding: true
enc_layers: 6
dec_layers: 6
heads: 8
hidden_size: 1024
word_vec_size: 1024
transformer_ff: 2048
dropout_steps: [0]
dropout: [0.3]
attention_dropout: [0.1]


'''

with open("en_gez.yaml", "w+") as config_yaml:
  config_yaml.write(config)

!cat en_gez.yaml


## Where the samples will be written
save_data: data/

## Where the vocab(s) will be written
src_vocab: data/en.vocab.src
tgt_vocab: data/gez.vocab.tgt

# Corpus opts:
data:
    corpus_1:
        path_src: data/en_train.txt
        path_tgt: data/gez_train.txt
    valid:
        path_src: data/en_val.txt
        path_tgt: data/gez_val.txt


## Where the model will be saved
save_model: model/model

# Prevent overwriting existing files in the folder
overwrite: True


# General opts
save_model: mybasemodel
save_checkpoint_steps: 500
valid_steps: 500
train_steps: 20000
seed: 3435

# Batching
bucket_size: 262144
world_size: 1
gpu_ranks: [0]
num_workers: 2
batch_type: "tokens"
batch_size: 1024
valid_batch_size: 128
accum_count: [4]
accum_steps: [0]

# Optimization
model_dtype: "fp16"
optim: "adam"
learning_rate: 0.1
warmup_steps: 1000
decay_method: "noam"
adam_beta2: 0.998
max_grad_norm: 0
label_smoothing: 0.1
param_init: 0
param_init_glorot: true
normalization: "tokens"

early_stopping: 5


In [3]:
ls data

en.vocab      en_train.txt  gez.vocab.tgt  gez_val.txt
en.vocab.src  en_val.txt    gez_test.txt   pred_1000.txt
en_test.txt   gez.vocab     gez_train.txt


In [4]:
!onmt_build_vocab -config en_gez.yaml -n_sample -1 

Corpus corpus_1's weight should be given. We default it to 1 for you.
[2023-02-07 10:33:03,430 INFO] Counter vocab from -1 samples.
[2023-02-07 10:33:03,430 INFO] n_sample=-1: Build vocab on full datasets.
[2023-02-07 10:33:03,721 INFO] Counters src:12930
[2023-02-07 10:33:03,721 INFO] Counters tgt:26781
[2023-02-07 10:33:03,721 WARNING] path data/en.vocab.src exists, may overwrite...
[2023-02-07 10:33:03,727 WARNING] path data/gez.vocab.tgt exists, may overwrite...


In [4]:
!onmt_train -config en_gez.yaml

[2023-02-07 10:41:35,211 INFO] Missing transforms field for corpus_1 data, set to default: [].
[2023-02-07 10:41:35,211 WARNING] Corpus corpus_1's weight should be given. We default it to 1 for you.
[2023-02-07 10:41:35,211 INFO] Missing transforms field for valid data, set to default: [].
[2023-02-07 10:41:35,211 INFO] Parsed 2 corpora from -data.
[2023-02-07 10:41:35,211 INFO] Get special vocabs from Transforms: {'src': [], 'tgt': []}.
[2023-02-07 10:41:35,302 INFO] Building model...
[2023-02-07 10:41:38,495 INFO] NMTModel(
  (encoder): TransformerEncoder(
    (embeddings): Embeddings(
      (make_embedding): Sequential(
        (emb_luts): Elementwise(
          (0): Embedding(12936, 1024, padding_idx=1)
        )
        (pe): PositionalEncoding(
          (dropout): Dropout(p=0.3, inplace=False)
        )
      )
    )
    (transformer): ModuleList(
      (0): TransformerEncoderLayer(
        (self_attn): MultiHeadedAttention(
          (linear_keys): Linear(in_features=1024, out_

[2023-02-07 10:42:11,618 INFO] Step 50/20000; acc: 4.0; ppl: 22854.4; xent: 10.0; lr: 0.00001; sents:    7321; bsz:  931/ 531/37; 5624/3204 tok/s;     33 sec;
[2023-02-07 10:42:27,686 INFO] Step 100/20000; acc: 6.9; ppl: 13879.4; xent: 9.5; lr: 0.00001; sents:    7266; bsz:  933/ 529/36; 11609/6582 tok/s;     49 sec;
[2023-02-07 10:42:43,723 INFO] Step 150/20000; acc: 6.5; ppl: 9867.5; xent: 9.2; lr: 0.00001; sents:    6862; bsz:  936/ 525/34; 11668/6543 tok/s;     65 sec;
[2023-02-07 10:42:59,970 INFO] Step 200/20000; acc: 6.9; ppl: 6773.7; xent: 8.8; lr: 0.00002; sents:    7345; bsz:  930/ 531/37; 11443/6536 tok/s;     81 sec;
[2023-02-07 10:43:16,206 INFO] Step 250/20000; acc: 7.1; ppl: 4858.0; xent: 8.5; lr: 0.00002; sents:    7464; bsz:  930/ 530/37; 11460/6531 tok/s;     98 sec;
[2023-02-07 10:43:32,493 INFO] Step 300/20000; acc: 8.5; ppl: 3706.5; xent: 8.2; lr: 0.00003; sents:    6800; bsz:  934/ 527/34; 11472/6472 tok/s;    114 sec;
[2023-02-07 10:43:48,637 INFO] Step 350/20000

[2023-02-07 10:53:26,736 INFO] valid stats calculation and sentences rebuilding
                           took: 8.411766052246094 s.
[2023-02-07 10:53:26,737 INFO] Train perplexity: 587.649
[2023-02-07 10:53:26,737 INFO] Train accuracy: 24.4774
[2023-02-07 10:53:26,737 INFO] Sentences processed: 287342
[2023-02-07 10:53:26,737 INFO] Average bsz:  932/ 528/36
[2023-02-07 10:53:26,737 INFO] Validation perplexity: 2594.88
[2023-02-07 10:53:26,737 INFO] Validation accuracy: 20.5711
[2023-02-07 10:53:26,737 INFO] Model is improving acc: 19.216 --> 20.5711.
[2023-02-07 10:53:26,746 INFO] Saving checkpoint mybasemodel_step_2000.pt
[2023-02-07 10:53:46,193 INFO] Step 2050/20000; acc: 53.6; ppl:  38.7; xent: 3.7; lr: 0.00007; sents:    7015; bsz:  933/ 526/35; 6692/3775 tok/s;    728 sec;
[2023-02-07 10:54:02,408 INFO] Step 2100/20000; acc: 55.9; ppl:  35.0; xent: 3.6; lr: 0.00007; sents:    7183; bsz:  932/ 528/36; 11499/6516 tok/s;    744 sec;
[2023-02-07 10:54:18,658 INFO] Step 2150/20000; 

[2023-02-07 11:04:04,569 INFO] Step 3850/20000; acc: 94.2; ppl:   7.2; xent: 2.0; lr: 0.00005; sents:    7518; bsz:  927/ 531/38; 11432/6549 tok/s;   1346 sec;
[2023-02-07 11:04:20,818 INFO] Step 3900/20000; acc: 94.7; ppl:   6.9; xent: 1.9; lr: 0.00005; sents:    7720; bsz:  927/ 535/39; 11404/6581 tok/s;   1362 sec;
[2023-02-07 11:04:37,136 INFO] Step 3950/20000; acc: 95.1; ppl:   6.7; xent: 1.9; lr: 0.00005; sents:    7192; bsz:  933/ 529/36; 11438/6478 tok/s;   1379 sec;
[2023-02-07 11:04:53,446 INFO] Step 4000/20000; acc: 95.0; ppl:   6.7; xent: 1.9; lr: 0.00005; sents:    6810; bsz:  937/ 529/34; 11490/6481 tok/s;   1395 sec;
[2023-02-07 11:05:01,943 INFO] valid stats calculation and sentences rebuilding
                           took: 8.495865106582642 s.
[2023-02-07 11:05:01,945 INFO] Train perplexity: 85.8032
[2023-02-07 11:05:01,945 INFO] Train accuracy: 52.5834
[2023-02-07 11:05:01,945 INFO] Sentences processed: 574569
[2023-02-07 11:05:01,945 INFO] Average bsz:  931/ 529/3

[2023-02-07 11:14:33,864 INFO] Step 5650/20000; acc: 99.0; ppl:   5.1; xent: 1.6; lr: 0.00004; sents:    7257; bsz:  930/ 530/36; 11494/6551 tok/s;   1975 sec;
[2023-02-07 11:14:50,071 INFO] Step 5700/20000; acc: 99.0; ppl:   5.1; xent: 1.6; lr: 0.00004; sents:    7083; bsz:  933/ 528/35; 11519/6519 tok/s;   1992 sec;
[2023-02-07 11:15:06,234 INFO] Step 5750/20000; acc: 98.9; ppl:   5.0; xent: 1.6; lr: 0.00004; sents:    7005; bsz:  934/ 529/35; 11561/6544 tok/s;   2008 sec;
[2023-02-07 11:15:22,455 INFO] Step 5800/20000; acc: 98.6; ppl:   5.2; xent: 1.6; lr: 0.00004; sents:    7300; bsz:  932/ 531/36; 11489/6545 tok/s;   2024 sec;
[2023-02-07 11:15:38,662 INFO] Step 5850/20000; acc: 99.1; ppl:   5.0; xent: 1.6; lr: 0.00004; sents:    7410; bsz:  930/ 531/37; 11471/6549 tok/s;   2040 sec;
[2023-02-07 11:15:54,953 INFO] Step 5900/20000; acc: 99.1; ppl:   5.0; xent: 1.6; lr: 0.00004; sents:    7062; bsz:  932/ 529/35; 11445/6497 tok/s;   2056 sec;
[2023-02-07 11:16:11,152 INFO] Step 5950

[2023-02-07 11:25:17,348 INFO] Saving checkpoint mybasemodel_step_7500.pt
[2023-02-07 11:25:36,598 INFO] Step 7550/20000; acc: 99.3; ppl:   4.7; xent: 1.6; lr: 0.00004; sents:    7114; bsz:  932/ 526/36; 6786/3828 tok/s;   2638 sec;
[2023-02-07 11:25:52,714 INFO] Step 7600/20000; acc: 99.3; ppl:   4.7; xent: 1.6; lr: 0.00004; sents:    7129; bsz:  930/ 525/36; 11536/6517 tok/s;   2654 sec;
[2023-02-07 11:26:08,893 INFO] Step 7650/20000; acc: 99.6; ppl:   4.6; xent: 1.5; lr: 0.00004; sents:    7275; bsz:  931/ 530/36; 11505/6557 tok/s;   2670 sec;
[2023-02-07 11:26:25,191 INFO] Step 7700/20000; acc: 99.3; ppl:   4.7; xent: 1.5; lr: 0.00004; sents:    7538; bsz:  928/ 531/38; 11391/6511 tok/s;   2687 sec;
[2023-02-07 11:26:41,439 INFO] Step 7750/20000; acc: 99.2; ppl:   4.7; xent: 1.6; lr: 0.00004; sents:    7128; bsz:  933/ 528/36; 11482/6495 tok/s;   2703 sec;
[2023-02-07 11:26:57,686 INFO] Step 7800/20000; acc: 99.2; ppl:   4.7; xent: 1.5; lr: 0.00004; sents:    7301; bsz:  931/ 528/3

[2023-02-07 11:36:41,615 INFO] Step 9500/20000; acc: 99.7; ppl:   4.4; xent: 1.5; lr: 0.00003; sents:    7301; bsz:  927/ 528/37; 11406/6500 tok/s;   3303 sec;
[2023-02-07 11:36:50,001 INFO] valid stats calculation and sentences rebuilding
                           took: 8.38528823852539 s.
[2023-02-07 11:36:50,002 INFO] Train perplexity: 16.4817
[2023-02-07 11:36:50,002 INFO] Train accuracy: 79.3768
[2023-02-07 11:36:50,002 INFO] Sentences processed: 1.36264e+06
[2023-02-07 11:36:50,002 INFO] Average bsz:  932/ 528/36
[2023-02-07 11:36:50,002 INFO] Validation perplexity: 2674.93
[2023-02-07 11:36:50,002 INFO] Validation accuracy: 23.8513
[2023-02-07 11:36:50,002 INFO] Model is improving acc: 23.7292 --> 23.8513.
[2023-02-07 11:36:50,011 INFO] Saving checkpoint mybasemodel_step_9500.pt
[2023-02-07 11:37:09,336 INFO] Step 9550/20000; acc: 99.8; ppl:   4.4; xent: 1.5; lr: 0.00003; sents:    7091; bsz:  931/ 527/35; 6720/3803 tok/s;   3331 sec;
[2023-02-07 11:37:25,547 INFO] Step 9600/20

[2023-02-07 11:47:12,159 INFO] Step 11300/20000; acc: 99.9; ppl:   4.3; xent: 1.5; lr: 0.00003; sents:    7047; bsz:  933/ 528/35; 11522/6522 tok/s;   3934 sec;
[2023-02-07 11:47:28,422 INFO] Step 11350/20000; acc: 99.3; ppl:   4.4; xent: 1.5; lr: 0.00003; sents:    7180; bsz:  932/ 529/36; 11466/6505 tok/s;   3950 sec;
[2023-02-07 11:47:44,630 INFO] Step 11400/20000; acc: 99.9; ppl:   4.3; xent: 1.5; lr: 0.00003; sents:    7212; bsz:  931/ 530/36; 11484/6537 tok/s;   3966 sec;
[2023-02-07 11:48:00,846 INFO] Step 11450/20000; acc: 99.9; ppl:   4.3; xent: 1.5; lr: 0.00003; sents:    7170; bsz:  931/ 529/36; 11488/6526 tok/s;   3982 sec;
[2023-02-07 11:48:17,026 INFO] Step 11500/20000; acc: 99.9; ppl:   4.3; xent: 1.5; lr: 0.00003; sents:    7046; bsz:  935/ 530/35; 11552/6551 tok/s;   3999 sec;
[2023-02-07 11:48:25,334 INFO] valid stats calculation and sentences rebuilding
                           took: 8.30688190460205 s.
[2023-02-07 11:48:25,335 INFO] Train perplexity: 13.0814
[2023

[2023-02-07 11:57:41,324 INFO] Step 13100/20000; acc: 99.9; ppl:   4.2; xent: 1.4; lr: 0.00003; sents:    7070; bsz:  932/ 528/35; 11518/6530 tok/s;   4563 sec;
[2023-02-07 11:57:57,616 INFO] Step 13150/20000; acc: 99.9; ppl:   4.2; xent: 1.4; lr: 0.00003; sents:    7167; bsz:  930/ 533/36; 11422/6548 tok/s;   4579 sec;
[2023-02-07 11:58:14,270 INFO] Step 13200/20000; acc: 99.9; ppl:   4.2; xent: 1.4; lr: 0.00003; sents:    7163; bsz:  931/ 530/36; 11186/6367 tok/s;   4596 sec;
[2023-02-07 11:58:30,580 INFO] Step 13250/20000; acc: 99.9; ppl:   4.2; xent: 1.4; lr: 0.00003; sents:    7287; bsz:  930/ 530/36; 11407/6499 tok/s;   4612 sec;
[2023-02-07 11:58:46,863 INFO] Step 13300/20000; acc: 99.6; ppl:   4.3; xent: 1.5; lr: 0.00003; sents:    6975; bsz:  933/ 527/35; 11458/6477 tok/s;   4628 sec;
[2023-02-07 11:59:03,098 INFO] Step 13350/20000; acc: 99.9; ppl:   4.2; xent: 1.4; lr: 0.00003; sents:    7422; bsz:  930/ 531/37; 11454/6544 tok/s;   4645 sec;
[2023-02-07 11:59:19,355 INFO] Ste

[2023-02-07 12:08:59,406 INFO] Step 15050/20000; acc: 99.9; ppl:   4.2; xent: 1.4; lr: 0.00003; sents:    7251; bsz:  930/ 529/36; 6817/3877 tok/s;   5241 sec;
[2023-02-07 12:09:15,527 INFO] Step 15100/20000; acc: 99.9; ppl:   4.2; xent: 1.4; lr: 0.00003; sents:    7366; bsz:  930/ 529/37; 11543/6568 tok/s;   5257 sec;
[2023-02-07 12:09:31,748 INFO] Step 15150/20000; acc: 99.9; ppl:   4.2; xent: 1.4; lr: 0.00003; sents:    7322; bsz:  931/ 530/37; 11483/6533 tok/s;   5273 sec;
[2023-02-07 12:09:47,992 INFO] Step 15200/20000; acc: 99.9; ppl:   4.2; xent: 1.4; lr: 0.00003; sents:    7165; bsz:  932/ 527/36; 11471/6492 tok/s;   5289 sec;
[2023-02-07 12:10:04,332 INFO] Step 15250/20000; acc: 99.9; ppl:   4.2; xent: 1.4; lr: 0.00003; sents:    6993; bsz:  933/ 528/35; 11426/6468 tok/s;   5306 sec;
[2023-02-07 12:10:20,592 INFO] Step 15300/20000; acc: 99.9; ppl:   4.2; xent: 1.4; lr: 0.00003; sents:    7341; bsz:  930/ 526/37; 11436/6467 tok/s;   5322 sec;
[2023-02-07 12:10:36,870 INFO] Step

In [250]:
print('aman')

aman


In [251]:
!ls exp

In [200]:
cp mybasemodel_step_4500.pt bestmodel.pt

In [ ]:
ls bes

In [29]:
### Translate

!onmt_translate -model mybasemodel_step_13500.pt -src data/en_test.txt  -output data/pred_1000.txt -gpu 0 -verbose -beam_size 5

[2023-02-07 12:23:22,031 INFO] 
SENT 1: ['For', 'thou', 'art', 'my', 'hope,', 'O', 'Lord', '<unk>', 'thou', 'art', 'my', 'trust', 'from', 'my', 'youth.']
PRED 1: እስመ ኀለፈት የዋሃት እምኔነ ወተገሠጽነ።
PRED SCORE: -0.4516

[2023-02-07 12:23:22,031 INFO] 
SENT 2: ['But', 'as', 'for', 'you,', 'ye', 'thought', 'evil', 'against', 'me;', 'but', 'God', '<unk>', 'it', 'unto', 'good,', 'to', 'bring', 'to', 'pass,', 'as', 'it', 'is', 'this', 'day,', 'to', 'save', 'much', 'people', 'alive.']
PRED 2: አላ ክሕድክሙ ወተቈጥዐት ነፍስክሙ ኵነኔየ ከመ ኢትግበሩ ኀበ መንበረ እስራኤል እስከ ዮም በዛቲ ዕለት ።
PRED SCORE: -0.8347

[2023-02-07 12:23:22,031 INFO] 
SENT 3: ['And', 'he', 'put', 'the', 'golden', 'altar', 'in', 'the', 'tent', 'of', 'the', 'congregation', 'before', 'the', 'vail:']
PRED 3: ወአንበረ ማዕጠንተ ዘወርቅ ውስተ ደብተራ ዘመርጡል እምነ መንጦላዕት ።
PRED SCORE: -0.4929

[2023-02-07 12:23:22,031 INFO] 
SENT 4: ['Because', 'the', 'daughters', 'of', 'Manasseh', 'had', 'an', 'inheritance', 'among', 'his', 'sons:', 'and', 'the', 'rest', 'of', "Manasseh's", 'sons', 

[2023-02-07 12:23:22,469 INFO] 
SENT 31: ['They', 'give', 'drink', 'to', 'every', 'beast', 'of', 'the', 'field:', 'the', 'wild', 'asses', 'quench', 'their', '<unk>']
PRED 31: እዝነ ቦሙ ወኢይሰምዑ፤ አንፈ ቦሙ ወኢያጼንዉ።
PRED SCORE: -0.5990

[2023-02-07 12:23:22,469 INFO] 
SENT 32: ['Thine', 'eyes', 'did', 'see', 'my', 'substance,', 'yet', 'being', '<unk>', 'and', 'in', 'thy', 'book', 'all', 'my', '<unk>', 'were', '<unk>', 'which', 'in', '<unk>', 'were', '<unk>', 'when', 'as', 'yet', 'there', 'was', 'none', 'of', 'them.']
PRED 32: ወዘሂ ኢሰአልከኒ እሁበከ ብዕለ ወክብረ ከመ ኢኮነ ኵሉ ብእሲ ዘይገብር በፍኖት እንተ ባሕቲቱ ከመ ይመውት ።
PRED SCORE: -1.4218

[2023-02-07 12:23:22,469 INFO] 
SENT 33: ['These', 'are', 'they', 'that', 'were', 'numbered', 'by', 'Moses', 'and', 'Eleazar', 'the', 'priest,', 'who', 'numbered', 'the', 'children', 'of', 'Israel', 'in', 'the', 'plains', 'of', 'Moab', 'by', 'Jordan', 'near', 'Jericho.']
PRED 33: ወዝንቱ ውእቱ ኍለቊሆሙ ለሙሴ ወደቂቀ እልዓዛር ካህን ወደቂቀ ደቂቀ እስራኤል በአራቦት ዘሞአብ በማዕዶተ ዮርዳንስ ዘመንገለ ኢያሪኮ ።
PRED SCORE: -0.8280

[2

[2023-02-07 12:23:22,854 INFO] 
SENT 61: ['And', 'honey,', 'and', '<unk>', 'and', 'sheep,', 'and', '<unk>', 'of', 'kine,', 'for', 'David,', 'and', 'for', 'the', 'people', 'that', 'were', 'with', 'him,', 'to', 'eat:', 'for', 'they', 'said,', 'The', 'people', 'is', 'hungry,', 'and', 'weary,', 'and', 'thirsty,', 'in', 'the', 'wilderness.']
PRED 61: ወዝውእቱ አልባስ ዘይገብሩ መልበስተ እንግድዓ ወትብልያ ዘመቅፈርት ወጶዴሬ ወመልበስተ ቍጽሮ ድርማንቀ ዘብእሲ ዘሐቌ ወቅናተ ወትገብሩ ሎቱ ለአሮን አልባሲሁ ውስተ ሐቅል ለዋው በውስተ ገዳም ።
PRED SCORE: -1.3012

[2023-02-07 12:23:22,855 INFO] 
SENT 62: ['And', 'Reuben', 'heard', 'it,', 'and', 'he', 'delivered', 'him', 'out', 'of', 'their', 'hands;', 'and', 'said,', 'Let', 'us', 'not', 'kill', 'him.']
PRED 62: ወሰምዐ ሩቤል ምስለ አበዊሁ ወይቤሎሙ ምንተ ኮንኪ ።
PRED SCORE: -1.2747

[2023-02-07 12:23:22,855 INFO] 
SENT 63: ['And', 'they', 'departed', 'from', '<unk>', 'and', 'pitched', 'in', 'Libnah.']
PRED 63: ወግዕዙ እምነ ሶቆት ወኀደሩ ውስተ ቡታን ።
PRED SCORE: -0.6154

[2023-02-07 12:23:22,855 INFO] 
SENT 64: ['And', 'he', 'shall', 'take', 'aw

[2023-02-07 12:23:23,212 INFO] 
SENT 91: ['I', 'have', 'made', 'a', 'covenant', 'with', 'my', 'chosen,', 'I', 'have', 'sworn', 'unto', 'David', 'my', 'servant,']
PRED 91: ናሁ ሰማዕኩ አነ ከመ ይቅትሎ ለዳዊት ።
PRED SCORE: -0.9237

[2023-02-07 12:23:23,212 INFO] 
SENT 92: ['Now', 'therefore,', 'O', 'king,', 'come', 'down', 'according', 'to', 'all', 'the', 'desire', 'of', 'thy', 'soul', 'to', 'come', 'down;', 'and', 'our', 'part', 'shall', 'be', 'to', 'deliver', 'him', 'into', 'the', "king's", 'hand.']
PRED 92: ወገብአ ንጉሥ ወበጽሐ ኀበ ኵሉ ንዋይከ ኀቤየ ወወረደ ኀበ ዳዊት ወያስተሰሪ ሎቱ ንጉሥ ወከመ ያዕድውዎ ለንጉሥ ዮርዳኖስ ።
PRED SCORE: -1.5251

[2023-02-07 12:23:23,213 INFO] 
SENT 93: ['And', 'his', "mother's", 'brethren', 'spake', 'of', 'him', 'in', 'the', 'ears', 'of', 'all', 'the', 'men', 'of', 'Shechem', 'all', 'these', 'words:', 'and', 'their', 'hearts', 'inclined', 'to', 'follow', 'Abimelech;', 'for', 'they', 'said,', 'He', 'is', 'our', 'brother.']
PRED 93: ለእሙ ቦአ አኀዊሁ ለኵሉ ዕደወ ለኵሉ አዕሩገ እስራኤል ።
PRED SCORE: -1.2061

[2023-02-07 12:2

[2023-02-07 12:23:23,551 INFO] 
SENT 121: ['He', 'rebuked', 'the', 'Red', 'sea', 'also,', 'and', 'it', 'was', 'dried', 'up:', 'so', 'he', 'led', 'them', 'through', 'the', '<unk>', 'as', 'through', 'the', 'wilderness.']
PRED 121: ዘይኴንን በኀይሉ ዘለዓለም ወአዕይንቲሁኒ ኀበ አሕዛብ ይኔጽራ፤ እለ ታአምሩ ኢታዕብዩ ርእሰክሙ።
PRED SCORE: -1.0642

[2023-02-07 12:23:23,551 INFO] 
SENT 122: ['Thou', 'shalt', 'eat', 'it', 'before', 'the', 'LORD', 'thy', 'God', 'year', 'by', 'year', 'in', 'the', 'place', 'which', 'the', 'LORD', 'shall', 'choose,', 'thou', 'and', 'thy', 'household.']
PRED 122: ወብልዑ በህየ በቅድመ እግዚአብሔር አምላክከ አመ ዕለተ ዘኀርየ እግዚአብሔር አምላክከ ከመ ይሰመይ ስሙ በህየ ።
PRED SCORE: -0.5440

[2023-02-07 12:23:23,552 INFO] 
SENT 123: ['So', 'Absalom', 'dwelt', 'two', 'full', 'years', 'in', 'Jerusalem,', 'and', 'saw', 'not', 'the', "king's", 'face.']
PRED 123: ወአበሴሎምሰ ተኀጥአ ወሖረ ውስተ ኢየሩሳሌም ወርእዮ ለንጉሥ ።
PRED SCORE: -0.6558

[2023-02-07 12:23:23,552 INFO] 
SENT 124: ['And', 'the', 'LORD', 'shewed', 'signs', 'and', 'wonders,', 'great', 'and', '

...

[2023-02-07 12:23:53,787 INFO] 
SENT 2221: ['And', 'the', 'thing', 'pleased', 'the', 'children', 'of', 'Israel;', 'and', 'the', 'children', 'of', 'Israel', 'blessed', 'God,', 'and', 'did', 'not', '<unk>', 'to', 'go', 'up', 'against', 'them', 'in', 'battle,', 'to', 'destroy', 'the', 'land', 'wherein', 'the', 'children', 'of', 'Reuben', 'and', 'Gad', 'dwelt.']
PRED 2221: ወእኩየ ኮነ ዝንቱ ነገር ደቂቀ እስራኤል ወባረኮሙ ወይቤ ኢየሱስ ከመ ይዕረጉ ይትቃተልዎሙ ።
PRED SCORE: -1.3928

[2023-02-07 12:23:53,787 INFO] 
SENT 2222: ['Then', 'died', '<unk>', 'and', 'was', 'buried', 'at', 'Bethlehem.']
PRED 2222: ወሞተ አራን በኀበ ውስተ ቤተ ልሔም ።
PRED SCORE: -0.6209

[2023-02-07 12:23:53,787 INFO] 
SENT 2223: ['I', 'have', 'wounded', 'them', 'that', 'they', 'were', 'not', 'able', 'to', '<unk>', 'they', 'are', 'fallen', 'under', 'my', 'feet.']
PRED 2223: ወእቀጠቅጦሙ ኢይትነሥኡ እንከ ወይወድቁ ታሕተ እገርየ ።
PRED SCORE: -0.2996

[2023-02-07 12:23:53,787 INFO] 
SENT 2224: ['And', 'the', 'LORD', 'called', 'unto', 'Moses,', 'and', 'spake', 'unto', 'him', 'out',

[2023-02-07 12:23:54,148 INFO] 
SENT 2251: ['So', 'Joshua', 'sent', 'messengers,', 'and', 'they', 'ran', 'unto', 'the', '<unk>', 'and,', 'behold,', 'it', 'was', 'hid', 'in', 'his', 'tent,', 'and', 'the', 'silver', 'under', 'it.']
PRED 2251: ወለአከ ኢየሱስ ኀቤሃ ወይቤሎሙ ኦምጽእዎ በአራቱ ኀቤየ ወቅትልዎ ።
PRED SCORE: -1.1576

[2023-02-07 12:23:54,148 INFO] 
SENT 2252: ['And', 'Salah', 'lived', 'after', 'he', 'begat', 'Eber', 'four', 'hundred', 'and', 'three', 'years,', 'and', 'begat', 'sons', 'and', 'daughters.']
PRED 2252: ወሐይወ አርፋክሳድ እምድኅረ ወለዶ ለቃያናን ፬፻ወ፴ዓመተ ወወለደ ደቀ ወአዋልደ ወሞተ ። ወሐይወ ቃያናን ፻፴ዓመተ ወወለዶ ለሳላ ። ወሐይወ ቃያናን እምድኅረ ወለዶ ለሳላ ፬፻ወ፴ዓመተ ወወለደ ደቀ ወአዋልደ ወሞተ ።
PRED SCORE: -0.0490

[2023-02-07 12:23:54,148 INFO] 
SENT 2253: ['And', 'this', 'shall', 'be', 'your', 'north', 'border:', 'from', 'the', 'great', 'sea', 'ye', 'shall', 'point', 'out', 'for', 'you', 'mount', 'Hor:']
PRED 2253: ወዝንቱ ውእቱ ዘትበልዑ እምነ ማዕዶተ ዮርዳንስ ይትኌለቍ ለክሙ በቅድመ ኵሉ ትዕይንት ።
PRED SCORE: -0.9004

[2023-02-07 12:23:54,148 INFO] 
SENT 2254: ['So', 'the

[2023-02-07 12:23:54,540 INFO] 
SENT 2281: ['And', 'it', 'shall', 'come', 'to', 'pass,', 'when', 'Pharaoh', 'shall', 'call', 'you,', 'and', 'shall', 'say,', 'What', 'is', 'your', 'occupation?']
PRED 2281: ወሶበ ርእየ ፈርዖን ለኵሉ ፈርዖን ወይቤሎሙ ምንት ተግባርክሙ ፤
PRED SCORE: -0.9812

[2023-02-07 12:23:54,540 INFO] 
SENT 2282: ['Ye', 'shall', 'not', 'do', 'after', 'all', 'the', 'things', 'that', 'we', 'do', 'here', 'this', 'day,', 'every', 'man', 'whatsoever', 'is', 'right', 'in', 'his', 'own', 'eyes.']
PRED 2282: ወዕቀቡ ኵሎ ትእዛዞ ዘአነ እኤዝዘከ ዮም ከመ ትግበርዎ ።
PRED SCORE: -0.7750

[2023-02-07 12:23:54,540 INFO] 
SENT 2283: ['And', 'they', 'rose', 'up', 'early', 'in', 'the', 'morning,', 'and', 'gat', 'them', 'up', 'into', 'the', 'top', 'of', 'the', 'mountain,', 'saying,', 'Lo,', 'we', 'be', 'here,', 'and', 'will', 'go', 'up', 'unto', 'the', 'place', 'which', 'the', 'LORD', 'hath', 'promised:', 'for', 'we', 'have', 'sinned.']
PRED 2283: ወተንሥአ በጽባሕ በለዓም በጽባሕ ወቀበርዎ ውስተ ደብር ውስተ ርእሰ ደብር ወይቤሎሙ ሑሩ ንሖር ኀበ ሀሎ ህየ እስመ ገብረ እግዚ

[2023-02-07 12:23:54,952 INFO] 
SENT 2311: ['So', 'the', 'ark', 'of', 'the', 'LORD', 'compassed', 'the', 'city,', 'going', 'about', 'it', 'once:', 'and', 'they', 'came', 'into', 'the', 'camp,', 'and', 'lodged', 'in', 'the', 'camp.']
PRED 2311: ወነበረት ታቦተ እግዚአብሔር ውስተ ሀገር ወገብረት ወበልዑ ወሰትዩ ዐቢይ ።
PRED SCORE: -0.9377

[2023-02-07 12:23:54,952 INFO] 
SENT 2312: ['And', 'afterwards', 'she', 'bare', 'a', 'daughter,', 'and', 'called', 'her', 'name', '<unk>']
PRED 2312: ወሶበ ጸልኣ ይእቲ ብእሲት ወሰመየት ስሞ ሳሚ ።
PRED SCORE: -1.0366

[2023-02-07 12:23:54,952 INFO] 
SENT 2313: ['All', 'the', 'kingdom', 'of', 'Og', 'in', 'Bashan,', 'which', 'reigned', 'in', 'Ashtaroth', 'and', 'in', 'Edrei,', 'who', 'remained', 'of', 'the', 'remnant', 'of', 'the', 'giants:', 'for', 'these', 'did', 'Moses', '<unk>', 'and', 'cast', 'them', 'out.']
PRED 2313: ኵሎ መንግሥቶ ለአግ ዘባሳን ዘነግሠ በኤድራይን ወበአስጣሮት ውእቱ ዘተርፈ እምነ ደሞሙ ለእለ ያርብሕ ወቀተሎ ሙሴ ወአጥፍኦ ወኢያጥፍእዎሙ ደቂቀ እስራኤል ለጌሲሪ ወለመካጢ ወለከናኔዎን እለ ውስቴቶሙ ወይነብር ንጉሠ ጌሲሪ ወመካጢ ውስተ ደቂቀ እስራኤል እስከ ዮም ወእስከ ዛቲ ዕለ

[2023-02-07 12:23:55,223 INFO] 
SENT 2341: ['And', 'the', 'coast', 'shall', 'go', 'down', 'from', '<unk>', 'to', '<unk>', 'on', 'the', 'east', 'side', 'of', '<unk>', 'and', 'the', 'border', 'shall', '<unk>', 'and', 'shall', 'reach', 'unto', 'the', 'side', 'of', 'the', 'sea', 'of', '<unk>', 'eastward:']
PRED 2341: ወኮነ ደወሎሙ እምነ ሊቦስ እስከ አሐዱ ኅብር ዘባሕረ አሊቄ እምነ ሀገር እንተ መንገለ ገቦሃ ለባሕረ አሊቄ ዘመንገለ ጽባሕ ።
PRED SCORE: -1.0087

[2023-02-07 12:23:55,223 INFO] 
SENT 2342: ['They', 'compassed', 'me', 'about', 'like', '<unk>', 'they', 'are', '<unk>', 'as', 'the', 'fire', 'of', '<unk>', 'for', 'in', 'the', 'name', 'of', 'the', 'LORD', 'I', 'will', 'destroy', 'them.']
PRED 2342: እዝነ ቦሙ ወኢይሰምዑ፤ አንፈ ቦሙ ወኢያጼንዉ።
PRED SCORE: -1.0234

[2023-02-07 12:23:55,223 INFO] 
SENT 2343: ['And', 'the', 'LORD', 'looked', 'upon', 'him,', 'and', 'said,', 'Go', 'in', 'this', 'thy', 'might,', 'and', 'thou', 'shalt', 'save', 'Israel', 'from', 'the', 'hand', 'of', 'the', 'Midianites:', 'have', 'not', 'I', 'sent', 'thee?']
PRED 234

In [30]:
gez_orignal = open('data/gez_test.txt')
ref = [gez_orignal.readlines()]
pred = open('data/pred_1000.txt', 'r')
sys = pred.readlines()

In [31]:
from sacrebleu.metrics import BLEU

In [32]:

bleu = BLEU()

bleu.corpus_score(sys, ref)

BLEU = 5.74 23.7/7.8/3.8/2.1 (BP = 0.930 ratio = 0.932 hyp_len = 30166 ref_len = 32361)